In [1]:
from ipyevents import Event 
##########
#########
import statistics
from ipywidgets import *
import io
import cv2
import PIL.Image
from scipy.ndimage import filters
import numpy as np
##########
from IPython.display import display, clear_output, IFrame, YouTubeVideo
from ipywebrtc import CameraStream, ImageRecorder
from itertools import chain
###########"
import webbrowser

In [2]:
def stop_video(_):
    out.layout.display = "none"
btn_exit_video = Button(description="Enlever la vidéo",tooltip="Cliquer pour enlever la vidéo",button_style='primary')
btn_exit_video.on_click(stop_video)
############
vid=YouTubeVideo('eWzY2nGfkXk')
# Box_Vid=HBox([vid,btn_exit_video])
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

with out:
    display(vid)
    display(btn_exit_video)
out.layout.display = "none"  #.layout.visibility = 'hidden'

Output(layout=Layout(border='1px solid black'))

In [3]:
#############################################
def adjust_brightness(img, brightness_factor):
    """Adjust brightness of an Image.
    Args:
        img (numpy ndarray): numpy ndarray to be adjusted.
        brightness_factor (float):  How much to adjust the brightness. Can be
            any non negative number. 0 gives a black image, 1 gives the
            original image while 2 increases the brightness by a factor of 2.
    Returns:
        numpy ndarray: Brightness adjusted image.
    """
    table = np.array([ i*brightness_factor for i in range (0,256)]).clip(0,255).astype('uint8')
    if img.shape[2]==1:
        return cv2.LUT(img, table)[:,:,np.newaxis]
    else:
        return cv2.LUT(img, table) 
##################################################
####################################################
def cible_petite(img,xc,yc):
    if len(img.shape) > 2 and img.shape[2] == 4:
        #convert the image from RGBA2RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    xcmin=xc-3
    xcmax=xc+3
    ycmin=yc-3
    ycmax=yc+3
    for x in range(xcmin,xcmax):
        img[ycmin,x]=[0,0,255]
    for x in range(xcmin,xcmax):
        img[ycmax,x]=[0,0,255]

    for y in range(ycmin,ycmax):
        img[y,xcmin]=[0,0,255]
    for y in range(ycmin,ycmax):
        img[y,xcmax]=[0,0,255]
    return img
####################################################
def fonction_p(img, lower_test, upper_test,inverse,w,wh, Nb_sup_wh, mode,List_info_paquets,diam_imp,Aire_col_imp):
    global affiche_Aire,ListeNbColPlus
    Rep_paquet=Choix_Paquet_widget.value
    if Rep_paquet=="On":
        mode="tapis"
    else:
        mode="isole"
#     display("dans fonction p")
#     display(Aire_col_imp,diam_imp)
    #     diam_imp=round((Aire_col_imp/10)**(1/2.0627))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)  ##  *** Attention, en Jupyter Ipywidget camera cré un alpha chanel

    modif=adjust_brightness(img, gamma)
    img=modif
    hauteur, largeur = img.shape[:2]
    # sortie d'image avec contours
    image_contours = img.copy()
    # Détection des colonies 
    colors = ['colonies']
    for color in colors:
        # copie de l'image originale
        image_to_process = img.copy()
        # initialisation du compteur
        counter[color] = 0
        # Dépôt des marges (GBR) de détection des couleurs dans un NumPy arrays
        if color == 'colonies':
            lower = np.array(lower_test)
            upper = np.array(upper_test)
        if inverse==1:
            image_to_process = (255-image_to_process)
        else:
            ok=2
        # Recherche des couleurs spécifiées dans les marges dans un masque
        image_mask2 = cv2.cvtColor(image_to_process, cv2.COLOR_BGR2HSV)

        image_mask = cv2.inRange(image_to_process, lower, upper)
        # application du masque
        image_res = cv2.bitwise_or(image_to_process, image_to_process, mask = image_mask)
        ## Transformation de l'image en nuance de gris et flouttage Gaussien
        image_gray = cv2.cvtColor(image_res, cv2.COLOR_BGR2GRAY)
        image_gray = cv2.GaussianBlur(image_gray, (5, 5), 0) # (7, 7)
        # Détection des contours, et résolution des imperfections
        image_edged = cv2.Canny(image_gray, 50, 100)
        image_edged = cv2.dilate(image_edged, None, iterations=1)
        image_edged = cv2.erode(image_edged, None, iterations=1)
        
        if mode=="isole":
            cnts, hierarchy = cv2.findContours(image_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cv2.findContours(image_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            
        else:
            cnts, hierarchy = cv2.findContours(image_edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            cnts = cv2.findContours(image_edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            #cnts = cv2.findContours(image_edged.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
            ###cnts = cv2.findContours(image_edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            
            
            #cnts_tot = cv2.findContours(image_edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE) ##
            #cnts_tour = cv2.findContours(image_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            #cnts = cnts_tot - cnts_tour


            


        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    #####################################
        if mode=="tapis":
            if hierarchy is None:
                ko=2
            else:
                hierar = list(chain.from_iterable(hierarchy))
                lenhierar=len(hierar)
                listi=[]
                for xc in range(0, lenhierar):
                    xcv=hierar[xc]
                    xcvb=hierar[xc][3]
                    if xcvb==-1:
                        listi.append(xc)
    ####
                listi=sorted(listi, reverse=True)
                #print(listi)
                leny=len(listi)
                for a in range(0,leny):
                    u=listi[a]
                    del hierar[u]
                    del cnts[u]
    #####################################
        Aires=[]
        compte=1
        Aire_col_imp=int(10*(pow(diam_imp,2.0627)))  ##float
        Nb_sup_wh=0
        for c in cnts:
        
            # Si le contour n'est pas suffisamment large, l'ignorer : C'est un paramètre à modifier pour les tailles de colonies
            ###if Aire_col_imp<1:
                ###yes=2
            ###else:
            if cv2.contourArea(c) < w or cv2.contourArea(c)>wh:   ##1  #5
                if cv2.contourArea(c)>wh:
                    aire_compt=cv2.contourArea(c)
                    infos=(compte,aire_compt)
                    compte+=1
                    List_info_paquets.append(infos)
                    Nb_sup_wh=Nb_sup_wh+1
                    #print("Nb_sup_wh ligne 117=",Nb_sup_wh)
                continue

            if cv2.contourArea(c)>Aire_col_imp:
                aire_compt=cv2.contourArea(c)
                infos=(compte,aire_compt)
                compte+=1
                List_info_paquets.append(infos)
                Nb_sup_wh=Nb_sup_wh+1
                #print("Nb_sup_wh ligne 126=",Nb_sup_wh)
                #print("List_info_paquets ligne 127=",List_info_paquets)
##############
            Aire=cv2.contourArea(c)
            Aires.append(Aire)
            # Création de la coqie convexe du contour
            hull = cv2.convexHull(c)
            if color == 'colonies':
                # Marquer le contour en vert
                cv2.drawContours(image_contours,[hull],0,coul,1) #0,0,255 # 204,102,0
            counter[color] += 1
            # Décocher le commentaire ci-dessous pour écrire, sur l'image, les TAILLES de colonies détectées
            if affiche_Aire==1:
                display("kaka")
                cv2.putText(image_contours, "{:.0f}".format(cv2.contourArea(c)), (int(hull[0][0][0]), int(hull[0][0][1])), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 0), 2)
            else:
                ko=1
#                 display("Prout")
        Len_Aires=len(Aires)
        if Len_Aires>0:
            Aire_Moy= int(statistics.mean(Aires) )
        else:
            Aire_Moy=0
        Aires=[]
    # Décocher le commentaire ci-dessous pour l'enregistrement de l'image finalisée
    #cv2.imwrite("Capture_denombrement.jpg",image_contours)     ######
    image_contours = image_contours.astype (np.uint8)
    return image_contours , counter[color],Nb_sup_wh,Aire_Moy,List_info_paquets
#####################################################    
def fonction_m(img,x,y, inverse): 
    p_unmin=255
    p_deuxmin=255
    p_troismin=255
    p_unmax=0
    p_deuxmax=0
    p_troismax=0
    if inverse==1:
        img = (255-img)
    else:
        ok=2
    image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)      
    xpmin=x
    xpmax=x+6
    ypmin=y
    ypmax=y+6
    cible_img=image[xpmin:xpmax,ypmin:ypmax]
    for xp in range(xpmin,xpmax-1):
        for yp in range(ypmin,ypmax-1):
            p=img[xp,yp]
            Prep_pixel = np.uint8([[p]])
            hsv_pixel = cv2.cvtColor(Prep_pixel,cv2.COLOR_BGR2HSV)
            pi=hsv_pixel[0]
            p_un=pi[0][0]
            p_deux=pi[0][1]
            p_trois=pi[0][2]
            if p_un<p_unmin:
                p_unmin=p_un
            else:
                z=1
            if p_un>p_unmax:
                p_unmax=p_un
            else:
                z=1
            if p_deux<p_deuxmin:
                p_deuxmin=p_deux
            else:
                z=1
            if p_deux>p_deuxmax:
                p_deuxmax=p_deux
            else:
                z=1 
            if p_trois<p_troismin:
                p_troismin=p_trois
            else:
                z=1
            if p_trois>p_troismax:
                p_troismax=p_trois
            else:
                z=1 
    #lower_test=[p_unmin,25,25]
    #lower_test=[p_unmin,p_deuxmin,p_troismin] ## cool
    lower_test=[p_unmin-32,p_deuxmin,p_troismin]  ###
    #upper_test=[p_unmax,255,255]
    #upper_test=[p_unmax,p_deuxmax,p_troismax]  ## cool
    upper_test=[p_unmax+32,p_deuxmax,p_troismax]  ###
    #print("lower=",lower_test)
    #print("upper=",upper_test)
    return lower_test, upper_test
####################################################
############
def f(change):
    global Nb_sup_wh, List_info_paquets, x2, y2,a, cl , gamma,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,ListeNbColPlus,diam_imp,mode
    
#     a=change['new']
    
    Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
    
def Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode): 
#     display("Aire Travail 1  =")
#     display(Aire_col_imp,diam_imp)
    if Iris_ok==1:
        global Nb_sup_wh, List_info_paquets,ListeNbColPlus  #  , x2, y2,a
        im_in = PIL.Image.open(io.BytesIO(image_headline2.value))
        im_array = np.array(im_in)#[..., :3]
    #     ####
        mask_circle = np.zeros(im_array.shape[:2],np.uint8)
        cv2.circle(mask_circle,(320,240), a.value , 255, -1)
        new = cv2.bitwise_and(im_array, im_array, mask=mask_circle)   
    #########################################"
    #########################################
        img=new
    ##############
        img=cible_petite(img,x2,y2)
        lower_test,upper_test=fonction_m(img,y2, x2,inverse) # (img,x2, y2,inverse)
        img,comptage,Nb_sup_wh,Aire_Moy,List_info_paquets=fonction_p(img, lower_test, upper_test,inverse,w,wh,Nb_sup_wh, mode,List_info_paquets,diam_imp,Aire_col_imp)
    ###################
#         display("Aire Travail 2  =")
#         display(Aire_col_imp,diam_imp)
    ###################
        Texte1="On compte : "
        Texte2=" colonies."
        texte2b=" colonie."
#         if comptage<2:
#             Texte=str(Texte1)+str(comptage)+str(texte2b)
#         else:
#             Texte=str(Texte1)+str(comptage)+str(Texte2)
#         cv2.putText(img, Texte ,(5,350), font, 0.6, (9,183,255), 2); # 0,0,255
########################################
        if comptage<2:
            Texte=str(Texte1)+str(comptage)+str(texte2b)
        else:
            Texte=str(Texte1)+str(comptage)+str(Texte2)
#         Texte3="Calibration_couleur="
#         if inverse==1:
#             texte11="Milieu"
#         else:
#             texte11="Colonie"
#         Texte4=str(Texte3)+str(texte11)
#         if mode=="isole":
#             Texte9=str("isolees")
#         else:
#             Texte9=str("rapprochees")
#         Texte8="Mode : "+Texte9
        Taille_limit=int(w/150)
        #Texte10=str(w)+" < Aire de 'colonie(s)' comptee(s) < "+str(wh)
        diam_col_inf=round((w/10)**(1/2.0627))
        diam_col_sup=round((wh/10)**(1/2.0627))
########################################
        if Aire_col_imp>0:
            #print("Aire_col_imp ligne 678=",Aire_col_imp)
            Texte23="Aire imposee ="+str(Aire_col_imp)
            #print("Aire imposee ligne684=",Aire_col_imp)
            #print(type(Aire_col_imp))
            cv2.putText(img, Texte23 ,(5,54), font, 2, (33,255,252), 2);
            ListeAirePaquet=[]
            ################

            NbColIsol=comptage-Nb_sup_wh
            #print("Nb_sup_wh ligne 690=",Nb_sup_wh )
            for paquet in range (0,Nb_sup_wh):
                #print("paquet=",paquet)
                #print("List_info_paquets=")
                #print(List_info_paquets)
                Num_paquet=List_info_paquets[paquet][0]
                #print("Num_paquet=",Num_paquet)
                Aire_paquet=List_info_paquets[paquet][1]
                #print("Aire_paquet=",Aire_paquet)   
                Aire_col_imp_elev=float(Aire_col_imp+((23/100)*Aire_col_imp))  ##float
                #print("Aire_col_imp_elev=",Aire_col_imp_elev)
                Nb_col=round(int(Aire_paquet/Aire_col_imp_elev),0)  ##float
                #Texte17="Paquet:"+str(Num_paquet)+", Nb.col="+str(Aire_paquet)
                Texte17="Paquet:"+str(Num_paquet)+", Nb.col="+str(Nb_col)
                ListeNbColPlus.append(Nb_col)
                hauteur=378+(12*Num_paquet)
                ListeAirePaquet.append(Aire_paquet)
#                 cv2.putText(img, Texte17 ,(5,hauteur), font, 2, (255,0,0), 2);

                #print("List_info_paquets=",List_info_paquets)
            ############
            #Texte23="Aire imposee ="+str(Aire_col_imp)
            #cv2.putText(img, Texte23 ,(5,180), font, 0.5, (0,0,255), 1);
            SomColPlus=int(sum(ListeNbColPlus))
            Texte21="Nb col. isolee:"+str(NbColIsol)
            #print("NbColIsol ligne 717=",NbColIsol)
            #print(type(NbColIsol))
            #Texte20="Nb col. paquets en +:"+str(SomColPlus)+"."
            Texte20="Nb col. paquets en +:"+str(SomColPlus)
            #print("Nb col. paquets en + ligne 721)",SomColPlus)
            NbTotcol=NbColIsol+SomColPlus  ##float
            Ni=NbTotcol
            Texte22="Nombre total de colonies: "+str(Ni)
            #Texte22="Nombre total de colonies: "+str(NbTotcol)
#             cv2.putText(img, Texte20 ,(5,340), font, 2, (255,0,0), 2);
#             cv2.putText(img, Texte21 ,(5,315), font, 2, (255,0,0), 2);
            cv2.putText(img, Texte22 ,(5,80), font, 2, (33,255,252), 2);
            #cv2.putText(img, "{:.0f}".format(Texte22) ,(5,93), font, 0.5, (0,0,255), 1);


            ListeNbColPlus=[]
            Aire_col_imp=float(10*(pow(diam_imp,2.0627)))  ##float
            #print("diam_imp=",diam_imp)
            #print("Aire_col_imp de diamp_imp=",Aire_col_imp)
            SomeListeAirePaquet=sum(ListeAirePaquet)
            #SommColTot=round(float(sum(SomeListeAirePaquet/Aire_col_imp)),1)
            if Aire_col_imp==0:
                Aire_col_imp=1
            SommColTot=float((SomeListeAirePaquet/Aire_col_imp)*(75/100))  ##float
            #Texte18="Nb colonie(s) deduite(s) en + ="+str(SommColTot)
            #cv2.putText(img, Texte18 ,(5,410), font, 0.5, (255,255,255), 1);
            #############

            ##########
            Nb_sup_wh=0
        List_info_paquets=[]
#         cv2.putText(img, Texte8 ,(550,30), font, 2, (255,255,255), 2);
#         cv2.putText(img, Texte4 ,(5,30), font, 2, (255,255,255), 2);
        cv2.putText(img, Texte ,(5,28), font, 2, (255,255,255), 2);
########################################
    ##################
        new=img
#########################################
    else:
        im_in = PIL.Image.open(io.BytesIO(image_headline2.value))
        im_array = np.array(im_in)  
        new=im_array
#########################################"
    fmt = 'png'
    data_comp = compress_to_bytes(new, fmt)
    image_headline.value=data_comp
    #####
    data_comp2 = compress_to_bytes(im_array, fmt)
    image_headline2.value=data_comp2
############
def g(b):
    im_in = PIL.Image.open(io.BytesIO(image_headline2.value))
    im_array = np.array(im_in)#[..., :3]

    img_rgb = cv2.flip(im_array, 1)

    mask_circle = np.zeros(img_rgb.shape[:2],np.uint8)
    cv2.circle(mask_circle,(320,240), a.value, 255, -1)
    new = cv2.bitwise_and(img_rgb, img_rgb, mask=mask_circle)  
    fmt = 'png'
    data_comp = compress_to_bytes(new, fmt)
    image_headline.value=data_comp
    #####
    data_comp = compress_to_bytes(img_rgb, fmt)
    image_headline2.value=data_comp
            #################################
def Recup_image(_):
    global Iris_ok
    Iris_ok=1
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)#[..., :3]
    ####
    mask_circle = np.zeros(im_array.shape[:2],np.uint8)
    cv2.circle(mask_circle,(320,240), a.value, 255, -1)
    img_rgb = cv2.bitwise_and(im_array, im_array, mask=mask_circle)    
    ####
    fmt = 'png'
    data_comp = compress_to_bytes(img_rgb, fmt)
    image_headline.value = data_comp
#     print("8888888888888888")
    ####
    data_comp2 = compress_to_bytes(im_array, fmt)
    image_headline2.value = data_comp2
                #################################
def compress_to_bytes(data, fmt):
    """
    Helper function to compress image data via PIL/Pillow.
    """
    buff = io.BytesIO()
    img = PIL.Image.fromarray(data)    
    img.save(buff, format=fmt)
    return buff.getvalue()
##########
def arrete_tout(_):
    Widget.close_all()
def update_coords(event):
    global Nb_sup_wh, List_info_paquets, x2, y2,a, cl , gamma,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,ListeNbColPlus,diam_imp,mode
    x2 = event['dataX']
    y2 = event['dataY']

    Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
#################
#########################               
def Contrast(_):
    global gamma,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,ListeNbColPlus,diam_imp,mode
    cl=clair_z.value
    fo=sombre_z.value
    cl_new=cl_new_z.value
    fo_new=fo_new_z.value
    if cl>cl_new:
        if fo==0:
            cl_new+=1
            cl_new_z.value=cl_new #
        else:
            clair_z.value=cl_new
            fo_new=fo_new-1
            fo_new_z.value=fo_new
            sombre_z.value=fo_new

    if fo>fo_new:
        if cl==0:
            fo_new+=1
            fo_new_z.value=fo_new
        else:
            sombre_z.value=fo_new
            cl_new=cl_new-1
            cl_new_z.value=cl_new
            clair_z.value=cl_new

    if fo==0:
        gamma=float(cl/10)+1
    if cl==0:
        gamma=1-float(fo/10)
    Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
#########################
##################################
def Fonc_isol(change):
    global x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode
    aq = change['new']
    if aq=="La couleur du milieu":
        inverse=1
    else:
        inverse=0
    Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
#####################################
def Chang_mod(change):
    global Nb_sup_wh, List_info_paquets, x2, y2,a, cl , gamma,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,ListeNbColPlus,diam_imp,mode
    mod =change['new']
    if mod=="Demo":        
        out.layout.display = "block"
        Mode_w.value="Intro"
    if mod == 'Tuto':
        yo.layout.visibility = 'hidden'
        Iris_ok=1
        Choix_tuto.disabled=False
    elif mod == 'Intro':
        yo.layout.visibility = 'hidden'
        Choix_tuto.value='#'
        Choix_tuto.disabled=True
        Titr_intro="Img0_jpg.jpg"
        #####
        intro_img = cv2.imread(Titr_intro, cv2.IMREAD_COLOR)
        intro_img_rgb = cv2.cvtColor(intro_img, cv2.COLOR_BGR2RGB)
        fmt = 'png'
        intro_img_data_comp = compress_to_bytes(intro_img_rgb, fmt)
#         image_headline.value = tut_img_data_comp
        image_headline2.value = intro_img_data_comp
        Iris_ok=0
        Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,Aire_col_imp,diam_imp,mode)
    elif mod== 'Capture':
        yo.layout.visibility = 'visible'
    
###################################
def Chang_tuto(change):
    global Nb_sup_wh, List_info_paquets, x2, y2,a, cl , gamma,Iris_ok , coul,w,wh,affiche_Aire,Aire_col_imp,ListeNbColPlus,diam_imp,mode
    tut=change['new']
    if tut=='#':
        rien=1
    else:
        tut=int(tut)
        tut_num=tut-1
        #####
        List_titre_tut = ['calib3.jpg','test5.jpg','test4.jpg','calib1.jpg','Staph1.jpg','BleuViolet.jpg']
        Titr_tuto=List_titre_tut[tut_num]
        #####
        tut_img = cv2.imread(Titr_tuto, cv2.IMREAD_COLOR)
        tut_img_rgb = cv2.cvtColor(tut_img, cv2.COLOR_BGR2RGB)
        fmt = 'png'
        tut_img_data_comp = compress_to_bytes(tut_img_rgb, fmt)
    #     image_headline.value = tut_img_data_comp
        image_headline2.value = tut_img_data_comp
        Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
##################################
def Chang_coul(change):
    global Nb_sup_wh, List_info_paquets, x2, y2,a, cl , gamma,Iris_ok, coul,w,wh,affiche_Aire,Aire_col_imp,ListeNbColPlus,diam_imp,mode
    coul_letr=change['new']
    if coul_letr=="Noir":
        coul=(0,0,0)
    elif coul_letr=="Rouge":
        coul=(255,0,0)
    elif coul_letr=="Vert":
        coul=(0,255,0)
    elif coul_letr=="Bleu":
        coul=(0,0,255)
    Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
###############################
def diam_inf_fonc(change):  
    global x2,y2,a,cl,gamma,inverse,Iris_ok,coul,wh,w,affiche_Aire,Aire_col_imp,ListeNbColPlus,diam_imp,mode
    diam_col_inf= change['new']
    w=float(10*(pow(diam_col_inf,2.0627)))
    Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
def diam_sup_fonc(change):  
    global x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,ListeNbColPlus,diam_imp,mode
    diam_col_sup= change['new']
    wh=float(10*(pow(diam_col_sup,2.0627)))
    Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
########################################"
# affiche_Aire=cv2.getTrackbarPos("Affich=", "Controle sur Aire imposee")
# Aire_col_imp=cv2.getTrackbarPos("Aire=", "Controle sur Aire imposee") 
##
# Affich_Air_w.observe(Affich_Air_w_fonc,names='value')
# Aire_col_imp_w = widgets.IntSlider(min = 0, max = 300,  value = 0 , description='Aire imposée')
# Aire_col_imp_w.observe(Aire_col_imp_fonc,names='value')
# Aire = VBox([Affich_Air_w,Aire_col_imp_w])

def Affich_Air_w_fonc(change):
    global x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh, affiche_Aire,Aire_col_imp,ListeNbColPlus,diam_imp,mode
    affiche_Aire_val=change['new']
#     display("dans affch aire")
#     display(Aire_col_imp,diam_imp)
    Aire_col_imp=Aire_col_imp_w.value
#     display("2eme affichage dans affich aire")
#     display(Aire_col_imp,diam_imp)
    if affiche_Aire_val==True:
        affiche_Aire=1
#         display("hello")
#         display(affiche_Aire)
        Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
    else:
        affiche_Aire=0
        Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
def Aire_col_imp_fonc(change):
    global x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh, affiche_Aire,ListeNbColPlus,diam_imp,diam_imp,mode
#     display("dans aire col imposee")
    Aire_col_imp=change['new']
#     display(Aire_col_imp,diam_imp)
    diam_imp=round((Aire_col_imp/10)**(1/2.0627))
#     display("apres calcul diam")
#     display(Aire_col_imp,diam_imp)
# #     display("Aire imp fonc =")
# #     display(Aire_col_imp)
    Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
#################################
def Fonc_Paquet(change):
    global x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp
    Rep_paquet=change['new']
    if Rep_paquet=="On":
        mode="tapis"
        Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
    else:
        mode="isole"
        Travail_img(x2,y2,a,cl,gamma,inverse,Iris_ok,coul,w,wh,affiche_Aire,Aire_col_imp,diam_imp,mode)
def Webbo(change):
    webbi=change['new']
    if webbi==True:
        webbrowser.open('https://biotech-online.pagesperso-orange.fr/Soutien%20TP/Iris.htm')
        Webbo_w.value=False
    else:
        rien=1

In [4]:
Iris_ok=0
affiche_Aire=0
#########################################
  # dictionnaire de comptage des colonies
counter = {}
coul=(255,0,0)
##################################
####### w,wh,Nb_sup_wh, mode,List_info_paquets,diam_imp ###
Aire_col_imp= 0    # int(10*(pow(diam_imp,2.0627)))  #60
diam_imp=round((Aire_col_imp/10)**(1/2.0627))
gamma=1
zoom=1
x = 306
y = 176
x2=274#240
y2=373#320
p_unmin=255
p_deuxmin=255
p_troismin=255
p_unmax=0
p_deuxmax=0
p_troismax=0
zoom2=0
rayon=150
Comptage_min=400
Comptage_max=0
#############
w=1
wh=1300  #500 #300
Nb_List=0
Nb_List_max=10
List_max=[]

Nb_List_min=0
List_min=[]
mode="isole"
Nb_sup_wh=0
List_info_paquets=[]
ListeNbColPlus=[]
cl=0
cl_new=0
fo=0
fo_new=0
weballum=0

#################""
font = cv2.FONT_HERSHEY_PLAIN  # FONT_HERSHEY_SIMPLEX  # FONT_HERSHEY_PLAIN
inverse=0
x2=274#240
y2=373#320
#######
clair_z = widgets.IntSlider(min = 0, max = 20,  value = 0 , description='+ clair')
sombre_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='+ sombre')
Lumino = Box([VBox([clair_z,sombre_z])])
Lumino.layout.border = 'thick double #00CC99' #00CC99
clair_z.observe(Contrast, names='value')
sombre_z.observe(Contrast, names='value')
###
cl_new_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='cl_new')
fo_new_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='fo_new')
# display(cl_new_z)
# display(fo_new_z)
# #####################################
diam_col_inf = widgets.IntSlider(min = 0, max = 11,  value = 0 , description='Min.')
diam_col_sup = widgets.IntSlider(min = 0, max = 11,  value = 11 , description='Max.')
diam_col_inf.observe(diam_inf_fonc,names='value')
diam_col_sup.observe(diam_sup_fonc,names='value')
titre_diam = Label("Diamètre de colonie :")
Diam = VBox([titre_diam,diam_col_inf,diam_col_sup])
Diam.layout.border = 'thick double #FF6633'#'3px solid black'
########################################
Webbo_w = widgets.ToggleButton(
    value=False,
    description='Site Web Iris',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour acéder au site Web Iris",
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
Webbo_w.observe(Webbo,names='value')
##################
Affich_Air_w = widgets.ToggleButton(
    value=False,
    description='Affiche Aire',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour voir l'aire des colonies",
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
Affich_Air_w.observe(Affich_Air_w_fonc,names='value')
Aire_col_imp_w = widgets.IntSlider(min = 0, max = 300,  value = 0 , description='Aire imposée')
Aire_col_imp_w.observe(Aire_col_imp_fonc,names='value')
Aire = VBox([Affich_Air_w,Aire_col_imp_w])
########################################
Calib_widget=widgets.RadioButtons(
            options=[
                "La couleur d'une colonie",
                "La couleur du milieu"
            ],
            layout={'width': 'max-content'}
        )
isol_z=widgets.Box(
    [
        widgets.Label(value='Calibration sur..'),
        Calib_widget
    ]
)
isol_z.layout.border = 'thick double #32a1ce'#'3px solid black'
Calib_widget.observe(Fonc_isol, names='value')
# #####################################
Choix_Paquet_widget=widgets.RadioButtons(
            options=[
                "On",
                "Off"
            ],
    value="Off",
            layout={'width': 'max-content'}
        )
Paquet_widget=widgets.Box(
    [
        widgets.Label(value='Résolution des Paquets :'),
        Choix_Paquet_widget
    ]
)
Paquet_widget.layout.border = 'thick double #32a1ce'#'3px solid black'
Choix_Paquet_widget.observe(Fonc_Paquet, names='value')
# #####################################
titre = cv2.imread("Titre.jpg", cv2.IMREAD_COLOR)
titre_rgb = cv2.cvtColor(titre, cv2.COLOR_BGR2RGB)
fmt = 'png'
titre_data_comp = compress_to_bytes(titre_rgb, fmt)
titre_w = widgets.Image(value=titre_data_comp,format='jpg',width='640')
# #####################################
img = cv2.imread("Img0_png.png", cv2.IMREAD_COLOR)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fmt = 'png'
data_comp = compress_to_bytes(img_rgb, fmt)
image_headline = widgets.Image(value=data_comp,format='jpg',width='640')
image_headline2 = widgets.Image(value=data_comp,format='jpg',width='640')
            #################################
im_events = Event()
im_events.source = image_headline
im_events.watched_events = ['click']
no_drag = Event(source=image_headline, watched_events=['dragstart'], prevent_default_action = True)
im_events.on_dom_event(update_coords)  # im_events.on_dom_event(update_coords)       
            #################################
Texte_w = Label('Bienvenue à tous dans Iris_V3 : La version en ligne !')
Texte_w.layout.border = '3px solid blue'


#             #################################        
a = widgets.IntSlider(min = 110, max = 238,  value = 150 , description="Taille de l'iris")
a.layout.width = '99%'
a.layout.border = 'thick double #330033'#'1px solid black'
# b = widgets.IntSlider(min = 0, max = 1,  value = 0 , description='b')

a.observe(f, names='value')
# a.visibility="none"
# b.observe(g, names='value')
            #################################
Choix_tuto = widgets.Dropdown(
    options=['#','1', '2', '3','4','5','6'],
    value='#',
    description='N° du Tuto :',
    disabled=True,
)
Mode_w=widgets.ToggleButtons(
    options=['Intro','Capture','Tuto','Demo'],
    value="Intro",
    description='Modes :',
    disabled=False,
    button_style='info',
)
Mod_Choi_Tuto = VBox([Mode_w,Choix_tuto])
Mod_Choi_Tuto.layout.border = 'thick double #00CCFF'
Mode_w.observe(Chang_mod,names='value')
Choix_tuto.observe(Chang_tuto, names='value')
################################
Coul_w=widgets.ToggleButtons(
    options=['Noir','Rouge','Vert','Bleu'],
    value="Rouge",
    description='Contours :',
    disabled=False,
    button_color = 'lightgreen', #button_style='primary', #success ### description='Custom color' # b1.style.button_color = 'lightgreen'
)
Coul_w.layout.border = 'thick double #33FF99'
Coul_w.observe(Chang_coul,names='value')
################################
btn_arret_appli = Button(description="Arreter l'appli",tooltip="Permet de sortir de l'application",button_style='danger')
btn_arret_appli.on_click(arrete_tout)
############
Resol_Paq=HBox([Aire,Paquet_widget])
Aire.layout.border ='1px solid blue'
Paquet_widget.layout.border ='1px solid blue'
Resol_Paq.layout.border = 'thick double #33FF99'
Fenetre_prin=HBox([ VBox([titre_w,image_headline,Texte_w]), VBox([Mod_Choi_Tuto,isol_z,a,Lumino,Coul_w,Diam,Resol_Paq,Webbo_w])  ])
Fenetre=VBox([btn_arret_appli,Fenetre_prin])
display(Fenetre)
#             #################################   


camera = CameraStream(constraints={'facing_mode': 'user','audio': False,'video': { 'width': 640, 'height': 480}})
image_recorder = ImageRecorder(stream=camera, tooltip='Cliquer pour tenter une Capture')
btn_stop = Button(description="Ok",tooltip='Cette capture est Ok ?',button_style='info')
btn_stop.on_click(Recup_image)

yo=HBox([camera,image_recorder, btn_stop])
yo.layout.visibility = 'hidden'
display(yo)


TypeError: Webbo() takes 0 positional arguments but 1 was given

In [5]:
# Widget.close_all()

In [6]:
# print(counter)